In [1]:
# This program is based on the project and its tutorial.
# https://huggingface.co/
# Dataset is from 
# https://www.kaggle.com/kritanjalijain/amazon-reviews

In [2]:
!pip install transformers
!pip install datasets
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"
!python -c "from datasets import load_dataset; print(load_dataset('squad', split='train')[0])"

     |████████████████████████████████| 2.6 MB 8.3 MB/s 
     |████████████████████████████████| 895 kB 49.5 MB/s 
     |████████████████████████████████| 3.3 MB 49.6 MB/s 
     |████████████████████████████████| 636 kB 58.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 8.3 MB/s 
     |████████████████████████████████| 118 kB 37.1 MB/s 
     |████████████████████████████████| 243 kB 38.6 MB/s 
     |████████████████████████████████| 76 kB 5.6 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
2021-08-12 10:09:32.095830: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Downloading: 100% 629/629 [00:00<00:00, 732kB/s]
Downloading: 100% 268M/268M [00:04<00:00, 55.8MB/

In [3]:
from google.colab import drive

drive.mount('/content/drive')
path = 'drive/My Drive/'

Mounted at /content/drive


In [4]:
!apt-get update
!pip3 install tqdm
import tqdm
print(tqdm.__version__)

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [5]:
# read dataset
import pandas as pd

df_train = pd.read_csv('drive/My Drive/amazon_review_polarity_csv/train.csv',names=('labels','key','text'))
df_train = df_train.dropna()

df_test = pd.read_csv('drive/My Drive/amazon_review_polarity_csv/test.csv',names=('labels','key','text'))
df_test = df_test.dropna()

In [6]:
df_train[0:4]

,labels,key,text
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,Amazing!,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...


In [7]:
# Colab(not pro)'s RAM is not enough. So we fine-tune the model with 0.1% data 
pos = int(len(df_train)*0.001)
df_train = df_train[0:pos]
#df_test = df_test[0:pos]

In [8]:
train_labels = df_train['labels'].values.tolist()
train_key = df_train['key'].values.tolist()

for i in range(len(train_labels)):
  if train_labels[i] == 2:
    train_labels[i] = 1
  else:
    train_labels[i] = 0


test_labels = df_test['labels'].values.tolist()
test_key = df_test['key'].values.tolist()

for i in range(len(test_labels)):
  if test_labels[i] == 2:
    test_labels[i] = 1
  else:
    test_labels[i] = 0

In [9]:
from sklearn.model_selection import train_test_split
train_key, eval_key, train_labels, eval_labels = train_test_split(train_key, train_labels, test_size=.2)

In [10]:
# preprocessing words->token

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_key, truncation=True, padding=True)
eval_encodings = tokenizer(eval_key, truncation=True, padding=True)
test_encodings = tokenizer(test_key, truncation=True, padding=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

In [11]:
# Build dataset same as content in toturial in huggingface

import torch

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, train_labels)
eval_dataset = MyDataset(eval_encodings, eval_labels)
test_dataset = MyDataset(test_encodings, test_labels)

In [12]:
# Train the model with training data(split train and validation)

import numpy as np
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    recall = recall_score(labels,predictions)
    f1 = f1_score(labels,predictions)
    precision = precision_score(labels,predictions)
    accuracy = accuracy_score(labels,predictions)
    result = {
        'accuracy':accuracy,
        'f1':f1,
        'precision':precision,
        'recall':recall
    }
    return result

training_args = TrainingArguments(
    output_dir='drive/My Drive/amazon_review_polarity_csv',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=100,
    weight_decay=0.01,
    evaluation_strategy='steps',
    save_strategy='epoch',
    seed=0,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.944800,0.876056,0.812500,0.791345,0.882759,0.717087
1000,0.896200,0.915700,0.787500,0.787204,0.781768,0.792717
1500,0.890500,0.843846,0.806944,0.820181,0.762019,0.887955
2000,0.826200,0.776234,0.826389,0.829001,0.810160,0.848739
2500,0.935600,0.894390,0.826389,0.829001,0.810160,0.848739
3000,0.787600,0.907731,0.830556,0.833787,0.811671,0.857143
3500,0.600600,0.823276,0.829167,0.816692,0.872611,0.767507
4000,0.517300,1.088186,0.812500,0.801762,0.842593,0.764706
4500,0.445000,1.129028,0.827778,0.818713,0.856269,0.784314
5000,0.460000,1.182313,0.806944,0.783826,0.881119,0.705882


***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
Saving model checkpoint to drive/My Drive/amazon_review_polarity_csv/checkpoint-2879
Configuration saved in drive/My Drive/amazon_review_polarity_csv/checkpoint-2879/config.json
Model weights saved in drive/My Drive/amazon_review_polarity_csv/checkpoint-2879/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
***** Running Evaluation *****
  Num examples = 720
  Batch size = 1
***** Running Evaluation *****
  

TrainOutput(global_step=8637, training_loss=0.5624700504085236, metrics={'train_runtime': 448.3172, 'train_samples_per_second': 19.265, 'train_steps_per_second': 19.265, 'total_flos': 248041840553316.0, 'train_loss': 0.5624700504085236, 'epoch': 3.0})

In [13]:
path = 'drive/My Drive/amazon_review_polarity_csv/checkpoint-8637'

model = DistilBertForSequenceClassification.from_pretrained(path,num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()


loading configuration file drive/My Drive/amazon_review_polarity_csv/checkpoint-8637/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "vocab_size": 30522
}

loading weights file drive/My Drive/amazon_review_polarity_csv/checkpoint-8637/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initia

{'eval_accuracy': 0.8130753268831721,
 'eval_f1': 0.809731270358306,
 'eval_loss': 1.17488431930542,
 'eval_precision': 0.8244896690039023,
 'eval_recall': 0.7954919323789856,
 'eval_runtime': 2373.3957,
 'eval_samples_per_second': 168.531,
 'eval_steps_per_second': 168.531}